In [12]:
from flask import Flask, request, render_template_string
import base64
import re
import cv2
import numpy as np
from threading import Thread

app = Flask(__name__)

In [13]:
html_page = """
<!DOCTYPE html>
<html>
<head>
  <title>Face Detection Camera</title>
</head>
<body>
  <h1 style="color: green;">Hello 👋</h1>
  <h2>Streaming camera with face detection</h2>

  <video id="video" width="640" height="480" autoplay></video>
  <canvas id="canvas" width="640" height="480" style="display:none;"></canvas>

  <script>
    const video = document.getElementById('video');
    const canvas = document.getElementById('canvas');
    const context = canvas.getContext('2d');

    navigator.mediaDevices.getUserMedia({ video: true })
      .then(stream => {
        video.srcObject = stream;
        setInterval(sendFrame, 500); // ส่งภาพทุก 0.5 วินาที
      })
      .catch(err => console.error("ไม่สามารถเข้าถึงกล้อง:", err));

    function sendFrame() {
      context.drawImage(video, 0, 0, canvas.width, canvas.height);
      const dataURL = canvas.toDataURL('image/jpeg');

      fetch('/upload', {
        method: 'POST',
        body: JSON.stringify({ image: dataURL }),
        headers: { 'Content-Type': 'application/json' }
      })
      .then(response => response.text())
      .then(data => console.log("Server response:", data))
      .catch(error => console.error("Error:", error));
    }
  </script>
</body>
</html>
"""

@app.route('/')
def serve_html():
    return render_template_string(html_page)



In [ ]:
#load model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def convert_to_base64(image):
    _, buffer = cv2.imencode('.jpg', image)
    image_base64 = base64.b64encode(buffer).decode('utf-8')
    return image_base64

@app.route('/upload', methods=['POST'])
def upload_image():
    # รับข้อมูล JSON จาก client
    data = request.get_json()
    image_data = data['image']

    # แปลงจาก base64 -> OpenCV image
    image_data = re.sub('^data:image/.+;base64,', '', image_data)
    image_bytes = base64.b64decode(image_data)
    np_arr = np.frombuffer(image_bytes, np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

    # ตรวจจับใบหน้า (Convert to grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # วาดกรอบรอบใบหน้า
    for (x, y, w, h) in faces:
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # แปลงภาพที่ตรวจจับแล้วกลับเป็น base64
    image_base64 = convert_to_base64(img)

    # ส่งภาพกลับไปยัง client
    return {'image': image_base64, 'faces_detected': len(faces)}


In [ ]:
def run_app():
    app.run(debug=False, use_reloader=False)

# รัน Flask server แบบ background thread
thread = Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/May/2025 11:26:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/May/2025 11:26:39] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:40] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:40] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:41] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:41] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:42] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:42] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:43] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:43] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:44] "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 -
127.0.0.1 - - [07/May/2025 11:26:44] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 11:26:4

In [6]:
import numpy as np
import urllib.request
import cv2

url = 'https://media.geeksforgeeks.org/wp-content/uploads/20211003151646/geeks14.png'

with urllib.request.urlopen(url) as resp:

    i = np.asarray(bytearray(resp.read()), dtype="uint8")
    
    i = cv2.imdecode(i, 0)
    
    cv2.imwrite("result.jpg", i)

In [12]:
!pip list

Package                        Version
------------------------------ --------------------
absl-py                        1.4.0
aiofiles                       22.1.0
aiohttp                        3.8.5
aiosignal                      1.2.0
aiosqlite                      0.18.0
altgraph                       0.17.4
anaconda-anon-usage            0.4.1
anaconda-catalogs              0.2.0
anaconda-client                1.12.0
anaconda-navigator             2.4.3
anaconda-project               0.11.1
anyio                          3.7.1
appdirs                        1.4.4
argon2-cffi                    21.3.0
argon2-cffi-bindings           21.2.0
array-record                   0.4.1
asgiref                        3.7.2
asttokens                      2.0.5
astunparse                     1.6.3
async-timeout                  4.0.2
attr                           0.3.1
attrs                          22.1.0
azure-core                     1.29.4
azure-storage-blob             12.18.3
Babel     

In [ ]:
#!pip install opencv-python
#!pip install opencv-contrib-python